In [2]:
import os
import glob
import pandas as pd 
import numpy as np
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

E:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df_shk = pd.read_csv('shahre_kord.csv')
df_shk = df_shk[df_shk.Ei != 0].reset_index(drop = True)
df_shk.Inci = df_shk.Inci / 10e5
df_shk = df_shk[df_shk.hhsi<5].reset_index(drop = True)
df_shk.Zone = list(range(1, df_shk.shape[0]+1))
y = df_shk['production']
X = df_shk.drop(['Zone', 'P-Wo', 'P-Sh','production'], axis = 1).reset_index(drop = True)
df_shk

,Zone,P-Wo,P-Sh,Ei,Ci,Inci,Sti,hhsi,production
0,1,182,390,300,0.265,2.943801,199,3.26,572
1,2,390,299,266,0.265,2.943801,127,3.18,689
2,3,1010,695,520,0.209,2.649290,309,3.15,1705
3,4,515,772,439,0.209,2.649290,273,3.08,1287
4,5,216,136,110,0.265,2.943801,67,3.45,352
...,...,...,...,...,...,...,...,...,...
56,57,2134,2109,1105,0.188,2.532398,661,3.29,4243
57,58,2189,1502,1167,0.188,2.532398,595,3.09,3691
58,59,1817,930,1058,0.188,2.532398,726,3.16,2747
59,60,2502,1752,1319,0.188,2.532398,793,3.24,4254


In [4]:
min_max_scaler = preprocessing.MinMaxScaler()

X_normalized = X.values #returns a numpy array
X_normalized = min_max_scaler.fit_transform(X_normalized)
X_normalized = pd.DataFrame(X_normalized)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=2)

In [6]:
# make sure to import all of our modules
# sklearn package
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# dataframes
import pandas as pd
# computation
import numpy as np
# visualization
import matplotlib.pyplot as plt

# PolynomialFeatures will create a new matrix consisting of all polynomial combinations 
# of the features with a degree less than or equal to the degree we just gave the model (2)
poly_model = PolynomialFeatures(degree=3)

# transform out polynomial features
poly_x_values = poly_model.fit_transform(x_train)

In [7]:
y_values = y_train
x_values = x_train

In [8]:
# let's fit the model
#poly_model.fit(poly_x_values, y_values)

# we use linear regression as a base!!! ** sometimes misunderstood **
regression_model = LinearRegression()
regression_model.fit(poly_x_values, y_values)
y_pred = regression_model.predict(poly_x_values)
y_pred = np.asarray(y_pred, dtype = 'int')

mean_squared_error(y_values, y_pred, squared=False)

0.5345224838248488

In [16]:
number_degrees = [1,2,3,4,5,6,7]
plt_mean_squared_error = []
for degree in number_degrees:

    poly_model = PolynomialFeatures(degree=degree)

    poly_x_values = poly_model.fit_transform(x_values)
    #poly_model.fit(poly_x_values, y_values)

    regression_model = LinearRegression()
    regression_model.fit(poly_x_values, y_values)
    y_pred = regression_model.predict(poly_x_values)

    plt_mean_squared_error.append(mean_squared_error(y_values, y_pred, squared=False))

In [17]:
min_mse = plt_mean_squared_error.index(min(plt_mean_squared_error)) + 1

In [18]:
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

poly = PolynomialFeatures(min_mse)
linr = LinearRegression()
model = make_pipeline(poly, linr)
model.fit(x_values, y_values)

coeff = model.steps[1][1].coef_
n_coeff = model.steps[0][1].get_feature_names()
data = pd.DataFrame(data={'name':n_coeff, 'coef':coeff})
data

E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,name,coef
0,1,-1.481867e-07
1,x0,-1.658687e+05
2,x1,-7.322925e+04
3,x2,-2.069130e+04
4,x3,2.680080e+05
5,x4,1.127043e+05
6,x0^2,2.280221e+04
7,x0 x1,1.043951e+05
8,x0 x2,-3.677416e+03
9,x0 x3,3.046023e+05


In [19]:
regression = LinearRegression()
regression.fit(x_values, y_values)
regression.score(x_test, y_test)

0.8221404317292357

In [20]:
regression.coef_

array([3208.7187917 ,  156.83155589,  -48.14026821, 2223.01329271,
        194.59532112])

In [21]:
regression.intercept_

141.04333256034806